In [ ]:
#https://pytorch.org/vision/stable/datasets.html#mnist
import torch
import torchvision
import torchvision.transforms as transforms

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
transform = transforms.ToTensor()
trainset = torchvision.datasets.MNIST('/tmp', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST('/tmp', train=False, download=True, transform=transform)


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(trainset.data[2])
print(trainset.data[2].shape)
print(device)

In [ ]:
BATCH_SIZE=4
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = x.view(BATCH_SIZE, -1)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x


In [ ]:
model = Model(28*28).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
loss_function = nn.CrossEntropyLoss()


In [ ]:
EPOCHS  = 100
import tqdm
for epoch in tqdm.trange(EPOCHS):
    for i, data in enumerate(trainloader):
      inputs, labels = data
      inputs, labels = Variable(inputs).to(device), Variable(labels).to(device)

      y_pred = model(inputs)
      loss = loss_function(y_pred, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      